<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Start-to-Finish Example: Gaussian pulse initial data for a massless scalar field, in Curvilinear coordinates

## Authors: Leonardo Werneck & Zach Etienne

# This tutorial notebook sets up initial data for a massless scalar field in *spherical-like coordinates*, using the *Numerical* ADM Spherical to BSSN Curvilinear initial data module (numerical = BSSN $\lambda^i$'s are computed using finite-difference derivatives instead of exact expressions).

**Notebook Status:** <font color='green'><b> Validated </b></font>

**Validation Notes:** This module has been validated to exhibit convergence to zero of the Hamiltonian constraint violation at the expected order to the exact solution (see [plots](#convergence) at bottom).</font>

## NRPy+ module to obtain the initial data used in this tutorial: **[ScalarFieldCollapse/ScalarField_InitialData.py](../edit/ScalarFieldCollapse/ScalarField_InitialData.py)**

## References

* [Tutorial-ADM_Initial_Data-ScalarField_GaussianPulse.ipynb](Tutorial-ADM_Initial_Data-ScalarField.ipynb)
* [Akbarian & Choptuik (2015)](https://arxiv.org/pdf/1508.01614.pdf) (Useful to understand the theoretical framework)
* [Baumgarte (2018)](https://arxiv.org/pdf/1807.10342.pdf) (Useful to understand the theoretical framework)
* [Baumgarte & Shapiro's Numerical Relativity](https://books.google.com.br/books/about/Numerical_Relativity.html?id=dxU1OEinvRUC&redir_esc=y): Section 6.2.2 (Useful to understand how to solve the Hamiltonian constraint)

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

* [Step 1](#set_nrpy_core): Set core NRPy+ parameters for numerical grids and reference metric
* [Step 2](#adm_id_scalar_field): Set up ADM initial data for the scalar field
* [Step 3](#adm_id_spacetime): Convert ADM initial data to BSSN-in-curvilinear coordinates
* [Step 4](#validate): Validating that the scalar field initial data satisfies the Hamiltonian constraint
    * [Step 4.a](#ham_const_output): Output the Hamiltonian constraint 
    * [Step 4.b](#bc_functs): Set up boundary condition functions for chosen singular, curvilinear coordinate system
    * [Step 4.c](#enforce3metric): Enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint
    * [Step 4.d](#cparams_rfm_and_domainsize): Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h`
* [Step 5](#mainc): `ScalarField_Playground.c`: The Main C Code
* [Step 6](#convergence): Validation: Convergence of numerical errors of the Scalar Field Collapse initial data (Hamiltonian constraint violation) to zero
* [Step 7](#latex_pdf_output): Output this module as $\LaTeX$-formatted PDF file

<a id='set_nrpy_core'></a>

# Step 1: Set core NRPy+ parameters for numerical grids and reference metric \[Back to [top](#toc)\]
$$\label{set_nrpy_core}$$

In [1]:
# Step P1: Import needed NRPy+ core modules:
from outputC import lhrh, add_to_Cfunction_dict  # NRPy+: Core C code output module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import grid as gri               # NRPy+: Functions having to do with numerical grids
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions

thismodule = "ScalarFieldCollapse_ID_setup"

# Step P2: Create C code output directory:
Ccodesrootdir = os.path.join("ScalarFieldID_Ccodes/")
# First remove C code output directory if it exists
# Courtesy https://stackoverflow.com/questions/303200/how-do-i-remove-delete-a-folder-that-is-not-empty
shutil.rmtree(Ccodesrootdir, ignore_errors=True)
# Then create a fresh directory
cmd.mkdir(Ccodesrootdir)

# Step P3: Create executable output directory:
outdir = os.path.join(Ccodesrootdir, "output")
cmd.mkdir(outdir)

# Step 1.a: Enable SIMD-optimized code?
#           I.e., generate BSSN and Ricci C code kernels using SIMD-vectorized
#           compiler intrinsics, which *greatly improve the code's performance*,
#           though at the expense of making the C-code kernels less
#           human-readable.
#           * Important note in case you wish to modify the BSSN/Ricci kernels
#             here by adding expressions containing transcendental functions
#             (e.g., certain scalar fields):
#           Note that SIMD-based transcendental function intrinsics are not
#           supported by the default installation of gcc or clang (you will
#           need to use e.g., the SLEEF library from sleef.org, for this
#           purpose). The Intel compiler suite does support these intrinsics
#           however without the need for external libraries.
enable_SIMD = True

# Step 1.b: Enable reference metric precomputation.
enable_rfm_precompute = True

if enable_SIMD and not enable_rfm_precompute:
    print("ERROR: SIMD does not currently handle transcendental functions,\n")
    print("       like those found in rfmstruct (rfm_precompute).\n")
    print("       Therefore, enable_SIMD==True and enable_rfm_precompute==False\n")
    print("       is not supported.\n")
    sys.exit(1)

# Step 1.c: Enable "FD functions". In other words, all finite-difference stencils
#         will be output as inlined static functions. This is essential for
#         compiling highly complex FD kernels with using certain versions of GCC;
#         GCC 10-ish will choke on BSSN FD kernels at high FD order, sometimes
#         taking *hours* to compile. Unaffected GCC versions compile these kernels
#         in seconds. FD functions do not slow the code performance, but do add
#         another header file to the C source tree.
# With gcc 7.5.0, enable_FD_functions=True decreases performance by 10%
enable_FD_functions = True

# Step 2: Set some core parameters, including CoordSystem MoL timestepping algorithm,
#                                 FD order, floating point precision, and CFL factor:
# Choices are: Spherical, SinhSpherical, SinhSphericalv2, Cylindrical, SinhCylindrical,
#              SymTP, SinhSymTP
CoordSystem     = "SinhSpherical"
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

# Step 2.a: Set outer boundary condition
# Current choices are extrapolation (quadratic polynomial extrapolation) or radiation
OuterBoundaryCondition = "radiation"
# OuterBoundaryCondition = "extrapolation"

# Step 2.b: Set defaults for Coordinate system parameters.
#           These are perhaps the most commonly adjusted parameters,
#           so we enable modifications at this high level.

# domain_size sets the default value for:
#   * Spherical's params.RMAX
#   * SinhSpherical*'s params.AMAX
#   * Cartesians*'s -params.{x,y,z}min & .{x,y,z}max
#   * Cylindrical's -params.ZMIN & .{Z,RHO}MAX
#   * SinhCylindrical's params.AMPL{RHO,Z}
#   * *SymTP's params.AMAX
domain_size     = 15 # Needed for all coordinate systems.

# sinh_width sets the default value for:
#   * SinhSpherical's params.SINHW
#   * SinhCylindrical's params.SINHW{RHO,Z}
#   * SinhSymTP's params.SINHWAA
sinh_width      = 0.2 # If Sinh* coordinates chosen

# sinhv2_const_dr sets the default value for:
#   * SinhSphericalv2's params.const_dr
#   * SinhCylindricalv2's params.const_d{rho,z}
sinhv2_const_dr = 0.05 # If Sinh*v2 coordinates chosen

# SymTP_bScale sets the default value for:
#   * SinhSymTP's params.bScale
SymTP_bScale    = 0.5 # If SymTP chosen

# Step 2.c: Set the order of spatial and temporal derivatives;
#           the core data type, and the CFL factor.
# RK_method choices include: Euler, "RK2 Heun", "RK2 MP", "RK2 Ralston", RK3, "RK3 Heun", "RK3 Ralston",
#              SSPRK3, RK4, DP5, DP5alt, CK5, DP6, L6, DP8
RK_method = "Euler"
FD_order  = 4            # Finite difference order: even numbers only, starting with 2. 12 is generally unstable
REAL      = "double"     # Best to use double here.
default_CFL_FACTOR= 0.5  # (GETS OVERWRITTEN IF SPECIFIED AT COMMAND LINE.)
                         # In pure axisymmetry (symmetry_axes = 2 below) 1.0 works fine. Otherwise 0.5 or lower.

In [2]:
# Step 5: Set the finite differencing order to FD_order (set above).
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", FD_order)

# Directory for reference_metric precomputation header files:
rfm_precompute_Ccode_outdir = os.path.join(Ccodesrootdir, "rfm_files/")
if enable_rfm_precompute:
    cmd.mkdir(os.path.join(Ccodesrootdir, "rfm_files/"))
    par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir", rfm_precompute_Ccode_outdir)

# Step 6: Copy SIMD/SIMD_intrinsics.h to $Ccodesrootdir/SIMD/SIMD_intrinsics.h
if enable_SIMD:
    cmd.mkdir(os.path.join(Ccodesrootdir,"SIMD"))
    shutil.copy(os.path.join("SIMD/")+"SIMD_intrinsics.h",os.path.join(Ccodesrootdir,"SIMD/"))

# Step 7: Set finite_difference::enable_FD_functions appropriately. Defaults to False
if enable_FD_functions:
    par.set_parval_from_str("finite_difference::enable_FD_functions", enable_FD_functions)

# Step 8: If enable_SIMD, then copy SIMD/SIMD_intrinsics.h to $Ccodesrootdir/SIMD/SIMD_intrinsics.h
cmd.mkdir(os.path.join(Ccodesrootdir,"SIMD"))
if enable_SIMD:
    shutil.copy(os.path.join("SIMD", "SIMD_intrinsics.h"), os.path.join(Ccodesrootdir, "SIMD"))

# Step 9: Set the direction=2 (phi) axis to be the symmetry axis; i.e.,
#         axis "2", corresponding to the i2 direction.
#      This sets all spatial derivatives in the phi direction to zero.
par.set_parval_from_str("indexedexp::symmetry_axes","12")
OMP_pragma_on = "i0"  # structure OpenMP loops to parallelize, not over i2 (phi direction), but i0 (radial direction)

In [3]:
# Step 10: Generate Runge-Kutta-based (RK-based) timestepping code.
#       As described above the Table of Contents, this is a 3-step process:
#       10.A: Evaluate RHSs (RHS_string)
#       10.B: Apply boundary conditions (post_RHS_string, pt 1)
#       10.C: Enforce det(gammabar) = det(gammahat) constraint (post_RHS_string, pt 2)
import MoLtimestepping.MoL_new_way as MoL

RHS_string      = ""
if OuterBoundaryCondition == "extrapolation":
    # Extrapolation BCs are applied to the evolved gridfunctions themselves after the MoL update
    post_RHS_string = "apply_bcs_curvilinear_extrapolation(params, bcstruct, NUM_EVOL_GFS, evol_gf_parity, griddata->xx, RK_OUTPUT_GFS);"
elif OuterBoundaryCondition == "radiation":
    # Radiation (Sommerfeld) BCs are applied to the gridfunction RHSs directly
    RHS_string += "apply_bcs_curvilinear_radiation(params, bcstruct, NUM_EVOL_GFS, evol_gf_parity, griddata->xx, RK_INPUT_GFS, RK_OUTPUT_GFS);"
    post_RHS_string = ""
else:
    print("Invalid choice of boundary condition")
    sys.exit(1)

post_RHS_string += ""

if not enable_rfm_precompute:
    RHS_string = RHS_string.replace("rfmstruct", "xx")
    post_RHS_string = post_RHS_string.replace("rfmstruct", "xx")

MoL.register_C_functions_and_NRPy_basic_defines(RK_method,
        RHS_string=RHS_string, post_RHS_string=post_RHS_string,
        enable_rfm=enable_rfm_precompute, enable_curviBCs=True, enable_SIMD=False, enable_griddata=True)

In [4]:
# Sets reference_metric globals: NRPy_basic_defines_str, rfm_struct__malloc, rfm_struct__define, rfm_struct__freemem
if enable_rfm_precompute:
    par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir", rfm_precompute_Ccode_outdir)
    par.set_parval_from_str("reference_metric::enable_rfm_precompute", "True")
    par.set_parval_from_str("reference_metric::rfm_precompute_to_Cfunctions_and_NRPy_basic_defines", "True")
    rfm.reference_metric()

rfm.register_C_functions(enable_rfm_precompute=enable_rfm_precompute, use_unit_wavespeed_for_find_timestep=True)
rfm.register_NRPy_basic_defines(enable_rfm_precompute=enable_rfm_precompute)

if enable_rfm_precompute:
    par.set_parval_from_str("reference_metric::enable_rfm_precompute", "False")
    rfm.ref_metric__hatted_quantities()

<a id='adm_id_scalar_field'></a>

# Step 2: Set up ADM initial data for the scalar field \[Back to [top](#toc)\]
$$\label{adm_id_scalar_field}$$

As documented [in the time-symmetric Gaussian pulse scalar field initial data NRPy+ tutorial notebook](Tutorial-ScalarField-Gaussian_pulse_initial_data-time_symmetric.ipynb), we generate the scalar field initial data and output it to a file.

Our initial data solver requires space matrices methods which are implemented in SciPy, so we make sure that it is installed.

In [5]:
!pip install scipy > /dev/null

Next we call the [`ScalarField.ScalarField_time_symmetric_initial_data.py()` function](../edit/ScalarField/ScalarField_time_symmetric_initial_data.py) ([NRPy+ tutorial notebook](Tutorial-ScalarField-Gaussian_pulse_initial_data-time_symmetric.ipynb)) to set up the initial data, using the default parameters for initial data. This function outputs the solution to a file named "outputSFID.txt".

In [6]:
import ScalarField.ScalarField_declare_gridfunctions as SFgfs
sf, sfM = SFgfs.declare_scalar_field_gridfunctions_if_not_declared_already()

In [7]:
# # Step 2.a: Import necessary Python and NRPy+ modules
# import ScalarField.ScalarField_InitialData as sfid

# # Step 2.b: Set the initial data parameters
# outputfilename      = os.path.join(outdir,"SFID.txt")
# initial_data_family = "Gaussian_pulse"
# pulse_amplitude     = 0.1
# pulse_center        = 0
# pulse_width         = 1
# Nr                  = 100000
# rmax                = domain_size*1.5

# # Step 2.c: Generate the initial data
# sfid.ScalarField_InitialData(outputfilename,initial_data_family,
#                              pulse_amplitude,pulse_center,pulse_width,Nr,rmax)

# # Step 2.d: Generate the needed C code
# sfid.NRPy_param_funcs_register_C_functions_and_NRPy_basic_defines(Ccodesdir=Ccodesdir,new_way=True)

<a id='adm_id_spacetime'></a>

# Step 3: Convert ADM initial data to BSSN-in-curvilinear coordinates \[Back to [top](#toc)\]
$$\label{adm_id_spacetime}$$

This is an automated process, taken care of by [`BSSN.ADM_Numerical_Spherical_or_Cartesian_to_BSSNCurvilinear`](../edit/BSSN.ADM_Numerical_Spherical_or_Cartesian_to_BSSNCurvilinear.py), and documented [in this tutorial notebook](Tutorial-ADM_Initial_Data-Converting_Numerical_ADM_Spherical_or_Cartesian_to_BSSNCurvilinear.ipynb).

In [8]:
# import BSSN.ADM_Numerical_Spherical_or_Cartesian_to_BSSNCurvilinear as AtoBnum
# AtoBnum.Convert_Spherical_or_Cartesian_ADM_to_BSSN_curvilinear("Spherical","ID_scalarfield_ADM_quantities",
#                                                                Ccodesdir=Ccodesdir,loopopts="")

<a id='validate'></a>

# Step 4: Validating that the scalar field initial data satisfies the Hamiltonian constraint \[Back to [top](#toc)\]
$$\label{validate}$$

We will validate that the scalar field initial data satisfies the Hamiltonian constraint, modulo numerical finite differencing error.

<a id='ham_const_output'></a>

## Step 4.a: Output the Hamiltonian constraint \[Back to [top](#toc)\]
$$\label{ham_const_output}$$

First output the Hamiltonian constraint [as documented in the corresponding NRPy+ tutorial notebook](Tutorial-BSSN_constraints.ipynb)

In [9]:
# # Enable rfm_precompute infrastructure, which results in
# #   BSSN RHSs that are free of transcendental functions,
# #   even in curvilinear coordinates, so long as
# #   ConformalFactor is set to "W" (default).
# cmd.mkdir(os.path.join(Ccodesdir,"rfm_files/"))
# par.set_parval_from_str("reference_metric::enable_rfm_precompute","True")
# par.set_parval_from_str("reference_metric::rfm_precompute_Ccode_outdir",os.path.join(Ccodesdir,"rfm_files/"))

# import BSSN.Enforce_Detgammahat_Constraint as EGC
# enforce_detg_constraint_symb_expressions = EGC.Enforce_Detgammahat_Constraint_symb_expressions()

# # Now register the Hamiltonian as a gridfunction.
# H = gri.register_gridfunctions("AUX","H")

# # Declare scalar field gridfunctions
# import ScalarField.ScalarField_declare_gridfunctions as sfgfs
# sf,sfM = sfgfs.declare_scalar_field_gridfunctions_if_not_declared_already()

# # Compute the scalar field energy-momentum tensor
# import ScalarField.ScalarField_Tmunu as sfTmunu
# sfTmunu.ScalarField_Tmunu()
# T4UU = sfTmunu.T4UU

# # Then define the Hamiltonian constraint and output the optimized C code.
# import BSSN.BSSN_constraints as bssncon
# import BSSN.BSSN_stress_energy_source_terms as Bsest
# bssncon.BSSN_constraints(add_T4UUmunu_source_terms=False)
# Bsest.BSSN_source_terms_for_BSSN_constraints(T4UU)
# bssncon.H += Bsest.sourceterm_H

# # Now that we are finished with all the rfm hatted
# #           quantities in generic precomputed functional
# #           form, let's restore them to their closed-
# #           form expressions.
# par.set_parval_from_str("reference_metric::enable_rfm_precompute","False") # Reset to False to disable rfm_precompute.
# rfm.ref_metric__hatted_quantities()

# desc="Evaluate the Hamiltonian constraint"
# name="Hamiltonian_constraint"
# outCfunction(
#     outfile  = os.path.join(Ccodesdir,name+".h"), desc=desc, name=name,
#     params   = """rfm_struct *restrict rfmstruct,const paramstruct *restrict params,
#                   REAL *restrict in_gfs, REAL *restrict auxevol_gfs, REAL *restrict aux_gfs""",
#     body     = fin.FD_outputC("returnstring",lhrh(lhs=gri.gfaccess("aux_gfs", "H"), rhs=bssncon.H),
#                               params="outCverbose=False"),
#     loopopts = "InteriorPoints,enable_rfm_precompute")

<a id='bc_functs'></a>

## Step 4.b: Set up boundary condition functions for chosen singular, curvilinear coordinate system \[Back to [top](#toc)\]
$$\label{bc_functs}$$

Next apply singular, curvilinear coordinate boundary conditions [as documented in the corresponding NRPy+ tutorial notebook](Tutorial-Start_to_Finish-Curvilinear_BCs.ipynb).

In [10]:
import CurviBoundaryConditions.CurviBoundaryConditions_new_way as CBC
CBC.CurviBoundaryConditions_register_C_functions_and_NRPy_basic_defines()
CBC.add_to_Cfunction_dict_set_up__bc_gz_map_and_parity_condns()
# We always need extrapolation, to fill in lambdaU data on the outer boundaries at t=0 (in most cases it's unknown there):
CBC.add_to_Cfunction_dict_apply_bcs_curvilinear(outer_bcs_type="extrapolation")
if OuterBoundaryCondition == "radiation":
    CBC.add_to_Cfunction_dict_apply_bcs_curvilinear(outer_bcs_type=OuterBoundaryCondition)

Evolved gridfunction "sf" has parity type 0.
Evolved gridfunction "sfM" has parity type 0.


<a id='enforce3metric'></a>

## Step 4.c: Enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint \[Back to [top](#toc)\]
$$\label{enforce3metric}$$

Then enforce conformal 3-metric $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint (Eq. 53 of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658)), as [documented in the corresponding NRPy+ tutorial notebook](Tutorial-BSSN_enforcing_determinant_gammabar_equals_gammahat_constraint.ipynb).

Applying curvilinear boundary conditions should affect the initial data at the outer boundary, and will in general cause the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint to be violated there. Thus after we apply these boundary conditions, we must always call the routine for enforcing the $\det{\bar{\gamma}_{ij}}=\det{\hat{\gamma}_{ij}}$ constraint:

In [11]:
# # Set up the C function for the det(gammahat) = det(gammabar)
# EGC.output_Enforce_Detgammahat_Constraint_Ccode(Ccodesdir,
#                                                 exprs=enforce_detg_constraint_symb_expressions)

<a id='cparams_rfm_and_domainsize'></a>

## Step 4.d: Output C codes needed for declaring and setting Cparameters; also set `free_parameters.h` \[Back to [top](#toc)\]
$$\label{cparams_rfm_and_domainsize}$$

Based on declared NRPy+ Cparameters, first we generate `declare_Cparameters_struct.h`, `set_Cparameters_default.h`, and `set_Cparameters[-SIMD].h`.

Then we output `free_parameters.h`, which sets initial data parameters, as well as grid domain & reference metric parameters, applying `domain_size` and `sinh_width`/`SymTP_bScale` (if applicable) as set above

In [12]:
# # Step 3.d.i: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
# par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

# # Step 3.d.ii: Set free_parameters.h
# # Output to $Ccodesdir/free_parameters.h reference metric parameters based on generic
# #    domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale,
# #    parameters set above.
# rfm.out_default_free_parameters_for_rfm(os.path.join(Ccodesdir,"free_parameters.h"),
#                                         domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale)

# # Step 3.d.iii: Generate set_Nxx_dxx_invdx_params__and__xx.h:
# rfm.set_Nxx_dxx_invdx_params__and__xx_h(Ccodesdir)

# # Step 3.d.iv: Generate xx_to_Cart.h, which contains xx_to_Cart() for
# #               (the mapping from xx->Cartesian) for the chosen
# #               CoordSystem:
# rfm.xx_to_Cart_h("xx_to_Cart","./set_Cparameters.h",os.path.join(Ccodesdir,"xx_to_Cart.h"))

# # Step 3.d.v: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
# par.generate_Cparameters_Ccodes(os.path.join(Ccodesdir))

In [13]:
# Step 3.e.i: Set free_parameters.h
outstr = r"""// Set free-parameter values.
// Set the default CFL Factor. Can be overwritten at command line.
REAL CFL_FACTOR = """+str(default_CFL_FACTOR)+r""";
"""

# Append to $Ccodesrootdir/free_parameters.h reference metric parameters based on generic
#    domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale,
#    parameters set above.
outstr += rfm.out_default_free_parameters_for_rfm("returnstring",
                                                  domain_size,sinh_width,sinhv2_const_dr,SymTP_bScale)
with open(os.path.join(Ccodesrootdir,"free_parameters.h"),"w") as file:
    file.write(outstr.replace("params.", "griddata.params."))

<a id='mainc'></a>

# Step 5: `main.c` \[Back to [top](#toc)\]
$$\label{mainc}$$

In [14]:
def add_to_Cfunction_dict_main__ScalarFieldID_Playground():
    includes = ["NRPy_basic_defines.h", "NRPy_function_prototypes.h", "time.h"]
    desc = """// main() function:
// Step 0: Read command-line input, set up grid structure, allocate memory for gridfunctions, set up coordinates
// Step 1: Set up initial data to an exact solution
// Step 2: Start the timer, for keeping track of how fast the simulation is progressing.
// Step 3: Integrate the initial data forward in time using the chosen RK-like Method of
//         Lines timestepping algorithm, and output periodic simulation diagnostics
// Step 3.a: Output 2D data file periodically, for visualization
// Step 3.b: Step forward one timestep (t -> t+dt) in time using
//           chosen RK-like MoL timestepping algorithm
// Step 3.c: If t=t_final, output conformal factor & Hamiltonian
//           constraint violation to 2D data file
// Step 3.d: Progress indicator printing to stderr
// Step 4: Free all allocated memory
"""
    c_type = "int"
    name = "main"
    params = "int argc, const char *argv[]"
    body = r"""
  griddata_struct griddata;
  set_Cparameters_to_default(&griddata.params);

  // Step 0.a: Set free parameters, overwriting Cparameters defaults
  //          by hand or with command-line input, as desired.
#include "free_parameters.h"

  // Step 0.b: Read command-line input, error out if nonconformant
  if((argc != 4 && argc != 5) || atoi(argv[1]) < NGHOSTS || atoi(argv[2]) < NGHOSTS || atoi(argv[3]) < 2 /* FIXME; allow for axisymmetric sims */) {
    fprintf(stderr,"Error: Expected three command-line arguments: ./BrillLindquist_Playground Nx0 Nx1 Nx2,\n");
    fprintf(stderr,"where Nx[0,1,2] is the number of grid points in the 0, 1, and 2 directions.\n");
    fprintf(stderr,"Nx[] MUST BE larger than NGHOSTS (= %d)\n",NGHOSTS);
    exit(1);
  }
  if(argc == 5) {
    CFL_FACTOR = strtod(argv[4],NULL);
    if(CFL_FACTOR > 0.5 && atoi(argv[3])!=2) {
      fprintf(stderr,"WARNING: CFL_FACTOR was set to %e, which is > 0.5.\n",CFL_FACTOR);
      fprintf(stderr,"         This will generally only be stable if the simulation is purely axisymmetric\n");
      fprintf(stderr,"         However, Nx2 was set to %d>2, which implies a non-axisymmetric simulation\n",atoi(argv[3]));
    }
  }
  // Step 0.c: Set up numerical grid structure, first in space...
  const int Nxx[3] = { atoi(argv[1]), atoi(argv[2]), atoi(argv[3]) };
  if(Nxx[0]%2 != 0 || Nxx[1]%2 != 0 || Nxx[2]%2 != 0) {
    fprintf(stderr,"Error: Cannot guarantee a proper cell-centered grid if number of grid cells not set to even number.\n");
    fprintf(stderr,"       For example, in case of angular directions, proper symmetry zones will not exist.\n");
    exit(1);
  }

  // Step 0.d: Uniform coordinate grids are stored to *griddata.xx[3]
  // Step 0.d.i: Set bcstruct
  {
    int EigenCoord = 1;
    // Step 0.d.ii: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
    //             params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
    //             chosen Eigen-CoordSystem.
    set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &griddata.params, griddata.xx);
    // Step 0.e: Find ghostzone mappings; set up bcstruct
    driver_bcstruct(&griddata.params, &griddata.bcstruct, griddata.xx);
    // Step 0.e.i: Free allocated space for xx[][] array
    for(int i=0;i<3;i++) free(griddata.xx[i]);
  }

  // Step 0f: Call set_Nxx_dxx_invdx_params__and__xx(), which sets
  //          params Nxx,Nxx_plus_2NGHOSTS,dxx,invdx, and xx[] for the
  //          chosen (non-Eigen) CoordSystem.
  int EigenCoord = 0;
  set_Nxx_dxx_invdx_params__and__xx(EigenCoord, Nxx, &griddata.params, griddata.xx);

  // Step 0.k: Declare struct for gridfunctions and allocate memory for y_n_gfs gridfunctions
  MoL_malloc_y_n_gfs(&griddata.params, &griddata.gridfuncs);
"""
    if enable_rfm_precompute:
        body += """
  // Step 0.l: Set up precomputed reference metric arrays
  // Step 0.l.i: Allocate space for precomputed reference metric arrays.
  rfm_precompute_rfmstruct_malloc(&griddata.params, &griddata.rfmstruct);

  // Step 0.l.ii: Define precomputed reference metric arrays.
  rfm_precompute_rfmstruct_define(&griddata.params, griddata.xx, &griddata.rfmstruct);"""
    body += r"""
  // Step 0.m: Set up initial data to an exact solution
  //initial_data(&griddata.params, griddata.xx, griddata.gridfuncs.y_n_gfs);
  
  // Step 0.n: Allocate memory for non-y_n_gfs gridfunctions
  MoL_malloc_non_y_n_gfs(&griddata.params, &griddata.gridfuncs);

  // Step 0.o: Apply boundary conditions, as initial data
  //          are sometimes ill-defined in ghost zones.
  //          E.g., spherical initial data might not be
  //          properly defined at points where r=-1.
  apply_bcs_curvilinear_extrapolation(&griddata.params, &griddata.bcstruct, NUM_EVOL_GFS,evol_gf_parity, griddata.xx, griddata.gridfuncs.y_n_gfs);
  //enforce_detgammahat_constraint(&griddata.params, &griddata.rfmstruct, griddata.gridfuncs.y_n_gfs);

  // Evaluate Hamiltonian constraint violation
  /*
  Hamiltonian_constraint(&rfmstruct, &params, y_n_gfs,auxevol_gfs, diagnostic_output_gfs);
  char filename[100];
  sprintf(filename,"out%d.txt",Nxx[0]);
  FILE *out1D = fopen(filename, "w");
  const int i1mid = Nxx_plus_2NGHOSTS1/2;
  const int i2mid = Nxx_plus_2NGHOSTS2/2;
  for(int i0=NGHOSTS;i0<Nxx_plus_2NGHOSTS0-NGHOSTS;i0++) {
    REAL xCart[3];
    xx_to_Cart(&params,xx,i0,i1mid,i2mid,xCart);
    int idx = IDX3S(i0,i1mid,i2mid);
    REAL rr = sqrt( xCart[0]*xCart[0] + xCart[1]*xCart[1] + xCart[2]*xCart[2] );
    fprintf(out1D,"%e %e\n",rr,log10(fabs(diagnostic_output_gfs[IDX4ptS(HGF,idx)])));
  }
  fclose(out1D);
  */

  // Step 4: Free all allocated memory
"""
    if enable_rfm_precompute:
        body += "  rfm_precompute_rfmstruct_freemem(&griddata.params, &griddata.rfmstruct);\n"
    body += r"""
  freemem_bcstruct(&griddata.params, &griddata.bcstruct);
  MoL_free_memory_y_n_gfs(&griddata.params, &griddata.gridfuncs);
  MoL_free_memory_non_y_n_gfs(&griddata.params, &griddata.gridfuncs);
  for(int i=0;i<3;i++) free(griddata.xx[i]);

  return 0;
"""
    # As rfmstruct stores functions of xx, when rfm_precompute is disabled,
    #   we always pass xx to a function instead of &rfmstruct.
    if not enable_rfm_precompute:
        body = body.replace("&rfmstruct", "xx")
    add_to_Cfunction_dict(
        includes=includes,
        desc=desc,
        c_type=c_type, name=name, params=params,
        body=body,
        rel_path_to_Cparams=os.path.join("."), enableCparameters=False)

In [15]:
add_to_Cfunction_dict_main__ScalarFieldID_Playground()

import outputC as outC
outC.outputC_register_C_functions_and_NRPy_basic_defines()  # #define M_PI,  etc.
# Declare paramstruct, register set_Cparameters_to_default(),
#   and output declare_Cparameters_struct.h and set_Cparameters[].h:
outC.NRPy_param_funcs_register_C_functions_and_NRPy_basic_defines(os.path.join(Ccodesrootdir))

gri.register_C_functions_and_NRPy_basic_defines()  # #define IDX3S(),  etc.
fin.register_C_functions_and_NRPy_basic_defines(NGHOSTS_account_for_onezone_upwind=True,
                                                enable_SIMD=enable_SIMD)  # #define NGHOSTS, and UPWIND() macro if SIMD disabled

# Output functions for computing all finite-difference stencils.
#   Must be called after defining all functions depending on FD stencils.
if enable_FD_functions:
    fin.output_finite_difference_functions_h(path=Ccodesrootdir)

# Call this last: Set up NRPy_basic_defines.h and NRPy_function_prototypes.h.
outC.construct_NRPy_basic_defines_h(Ccodesrootdir, enable_SIMD=enable_SIMD)
outC.construct_NRPy_function_prototypes_h(Ccodesrootdir)

In [16]:
import cmdline_helper as cmd
cmd.new_C_compile(Ccodesrootdir, os.path.join("output", "BrillLindquist_Playground"),
                  uses_free_parameters_h=True, compiler_opt_option="fast") # fastdebug or debug also supported

(EXEC): Executing `make -j34`...
(BENCH): Finished executing in 0.40246081352233887 seconds.
Finished compilation.


In [17]:
# os.chdir(outdir)
# cmd.delete_existing_files("out96.txt")
# cmd.Execute("ScalarField_Playground", "96 2 2", "out96.txt")
# cmd.delete_existing_files("out72.txt")
# cmd.Execute("ScalarField_Playground", "72 2 2", "out72.txt")
# cmd.delete_existing_files("out48.txt")
# cmd.Execute("ScalarField_Playground", "48 2 2", "out48.txt")
# os.chdir(os.path.join("..",".."))

<a id='convergence'></a>

# Step 6: Validation: Convergence of numerical errors (Hamiltonian constraint violation) to zero \[Back to [top](#toc)\]
$$\label{convergence}$$

The equations behind these initial data solve Einstein's equations exactly, at a single instant in time. One reflection of this solution is that the Hamiltonian constraint violation should be exactly zero in the initial data. 

However, when evaluated on numerical grids, the Hamiltonian constraint violation will *not* generally evaluate to zero due to the associated numerical derivatives not being exact. However, these numerical derivatives (finite difference derivatives in this case) should *converge* to the exact derivatives as the density of numerical sampling points approaches infinity.

In this case, all of our finite difference derivatives agree with the exact solution, with an error term that drops with the uniform gridspacing to the fourth power: $\left(\Delta x^i\right)^4$. 

Here, as in the [Start-to-Finish Scalar Wave (Cartesian grids) NRPy+ tutorial](Tutorial-Start_to_Finish-ScalarWave.ipynb) and the [Start-to-Finish Scalar Wave (curvilinear grids) NRPy+ tutorial](Tutorial-Start_to_Finish-ScalarWaveCurvilinear.ipynb) we confirm this convergence.

We demonstrate convergence along the radial direction in our 1D (spherically symmetric) problem.

In [18]:
# import numpy as np
# import matplotlib.pyplot as plt
# from IPython.display import Image

# r_96,log10H_96 = np.loadtxt(os.path.join(outdir,"out96.txt")).T
# r_72,log10H_72 = np.loadtxt(os.path.join(outdir,"out72.txt")).T
# r_48,log10H_48 = np.loadtxt(os.path.join(outdir,"out48.txt")).T

# # Set 4 subplots
# fig = plt.figure()

# plt.title("Plot Demonstrating 4th-order Convergence")
# plt.ylabel(r"$\log_{10}|H|$")
# plt.xlabel(r"$r$")
# plt.xlim(0,3)
# plt.ylim(-10,-5)
# plt.plot(r_96,log10H_96,color='black',ls='-',label=r"$N_{r}=96$")
# plt.plot(r_72,log10H_72+np.log10((72.0/96.0)**4),color='green',ls='-.',label=r"$N_{r}=72$, times (72/96)$^4$")
# plt.plot(r_48,log10H_48+np.log10((48.0/96.0)**4),color='red',ls='--',label=r"$N_{r}=60$, times (48/96)$^4$")
# legend = plt.legend(shadow=True,loc=3)
# legend.get_frame().set_facecolor('C1')
# plt.tight_layout()

# outfile = os.path.join(outdir,"HC_convergence_SFID.png")
# plt.savefig(outfile,facecolor='white',dpi=120)
# plt.close(fig)
# Image(outfile)

<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-Start_to_Finish-BSSNCurvilinear-Setting_up_ScalarField_initial_data.pdf](Tutorial-Start_to_Finish-BSSNCurvilinear-Setting_up_ScalarField_initial_data.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [19]:
# import cmdline_helper as cmd # NRPy+: Multi-platform Python command-line interface
# cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-Start_to_Finish-BSSNCurvilinear-Setting_up_ScalarField_initial_data")

In [25]:
def ScalarField_InitialData():
    from outputC import outputC, add_to_Cfunction_dict
    from sympy import Symbol
    from indexedexp import declarerank1, declarerank2
    from BSSN.ADM_Exact_Spherical_or_Cartesian_to_BSSNCurvilinear \
        import Convert_Spherical_or_Cartesian_ADM_to_BSSN_curvilinear
    from ScalarField.ScalarField_declare_gridfunctions \
        import declare_scalar_field_gridfunctions_if_not_declared_already

    sfSph, sfMSph = declare_scalar_field_gridfunctions_if_not_declared_already()
    alphaSph      = Symbol("alphaSph", real=True)
    betaSphU      = declarerank1("betaSphU")
    BSphU         = declarerank1("BSphU")
    gammaSphDD    = declarerank2("gammaSphDD", "sym01")
    KSphDD        = declarerank2("KSphDD"    , "sym01")
    cf,hDD,lambdaU,aDD,trK,alpha,vetU,betU = \
        Convert_Spherical_or_Cartesian_ADM_to_BSSN_curvilinear("Spherical", rfm.xxSph,
                                                               gammaSphDD,KSphDD,alphaSph,betaSphU,BSphU)

    
    includes = ["NRPy_basic_defines.h"]
    rhss = [sf, sfM, alpha, cf, trK]
    lhss = ["in_gfs[IDX4S(SFGF,i0,i1,i2)]",
            "in_gfs[IDX4S(SFMGF,i0,i1,i2)]",
            "in_gfs[IDX4S(ALPHAGF,i0,i1,i2)]",
            "in_gfs[IDX4S(CFGF,i0,i1,i2)]",
            "in_gfs[IDX4S(TRKGF,i0,i1,i2)]"]
    for i in range(3):
        rhss.append(lambdaU[i])
        lhss.append("in_gfs[IDX4S(LAMBDAU"+str(i)+"GF,i0,i1,i2)]")
        rhss.append(vetU[i])
        lhss.append("in_gfs[IDX4S(VETU"+str(i)+"GF,i0,i1,i2)]")
        rhss.append(betU[i])
        lhss.append("in_gfs[IDX4S(BETU"+str(i)+"GF,i0,i1,i2)]")
        for j in range(i,3):
            rhss.append(hDD[i][j])
            lhss.append("in_gfs[IDX4S(HDD" + str(i) + str(j) + "GF,i0,i1,i2)]")
            rhss.append(aDD[i][j])
            lhss.append("in_gfs[IDX4S(ADD" + str(i) + str(j) + "GF,i0,i1,i2)]")

    # Sort the lhss list alphabetically, and rhss to match:
    lhss,rhss = [list(x) for x in zip(*sorted(zip(lhss, rhss), key=lambda pair: pair[0]))]

    prefunc = """
static int count_num_lines_in_file(FILE *in1DSFID) {
  int numlines_in_file = 0;
  char * line = NULL;

  size_t len = 0;
  ssize_t read;
  while ((read = getline(&line, &len, in1DSFID)) != -1) {
    numlines_in_file++;
  }
  rewind(in1DSFID);

  free(line);
  return numlines_in_file;
}

static int read_datafile__set_arrays(FILE *in1DSFID,
                              REAL *r_arr,
                              REAL *sf_arr,
                              REAL *gammaDD00_or_psi4_arr,
                              REAL *alpha_arr) {
  char * line = NULL;

  size_t len = 0;
  ssize_t read;

  int which_line = 0;
  while ((read = getline(&line, &len, in1DSFID)) != -1) {
    // Define the line delimiters (i.e., the stuff that goes between the data on a given
    //     line of data.  Here, we define both spaces " " and tabs "\t" as data delimiters.
    const char delimiters[] = " \t";

    //Now we define "token", a pointer to the first column of data
    char *token;

    //Each successive time we call strtok(NULL,blah), we read in a new column of data from
    //     the originally defined character array, as pointed to by token.

    token                             = strtok(line, delimiters); if(token==NULL) { printf("BADDDD\n"); return 1; }
    r_arr[which_line]                 = strtod(token, NULL); token = strtok( NULL, delimiters );
    sf_arr[which_line]                = strtod(token, NULL); token = strtok( NULL, delimiters );
    gammaDD00_or_psi4_arr[which_line] = strtod(token, NULL); token = strtok( NULL, delimiters );
    alpha_arr[which_line]             = strtod(token, NULL); token = strtok( NULL, delimiters );

    which_line++;
  }
  free(line);
  return 0;
}

static int bisection_idx_finder(const REAL r_star, const int numlines_in_file, const REAL *r_arr) {
  int x1 = 0;
  int x2 = numlines_in_file-1;
  REAL y1 = r_star-r_arr[x1];
  REAL y2 = r_star-r_arr[x2];
  if(y1*y2 >= 0) {
    fprintf(stderr,"%d %d | %lf %lf | %lf %lf | %lf \n", x1, x2, y1, y2, r_arr[x1], r_arr[x2], r_star);
    fprintf(stderr,"INTERPOLATION BRACKETING ERROR %e | %e %e \n",r_star,y1,y2);
    exit(1);
  }
  for(int i=0;i<numlines_in_file;i++) {
    int x_midpoint = (x1+x2)/2;
    REAL y_midpoint = r_star-r_arr[x_midpoint];
    if(y_midpoint*y1 < 0) {
      x2 = x_midpoint;
      y2 = y_midpoint;
    } else {
      x1 = x_midpoint;
      y1 = y_midpoint;
    }
    if( abs(x2-x1) == 1 ) {
      // If r_arr[x1] is closer to r_star than r_arr[x2] then return x1:
      if(fabs(r_star-r_arr[x1]) < fabs(r_star-r_arr[x2])){
          return x1;
      }
      // Otherwiser return x2:
      return x2;
    }
  }
  fprintf(stderr,"INTERPOLATION BRACKETING ERROR: DID NOT CONVERGE.\n");
  exit(1);
}

static void scalarfield_interpolate_1D( REAL const  r_star_in,                       // Point of interest for interpolation
                                        int  const interp_stencil_size,              // Number of points used to interpolate
                                        int  const numlines_in_file,                 // Number of lines in initial data file
                                        REAL const *restrict r_arr,                  // Initial data array for r
                                        REAL const *restrict sf_arr,                 // Initial data array for varphi
                                        REAL const *restrict gammaDD00_or_psi4_arr,  // Initial data array for gamma_{rr}
                                        REAL const *restrict alpha_arr,              // Initial data array for alpha
                                        REAL       *restrict sf_star,                // "Output": varphi(t=0,r_star)
                                        REAL       *restrict gammaDD00_or_psi4_star, // "Output": gamma_{rr}(t=0,r_star) or psi^{4}(t=0,r_star)
                                        REAL       *restrict alpha_star) {           // "Output": alpha(t=0,r_star)
    
    
  // For this case, we know that for all functions, f(r) = f(-r)
  REAL r_star = r_star_in;
  if ( r_star < 0 ) r_star = -r_star;
    
  // First we find the central interpolation stencil index:
  int idx = bisection_idx_finder(r_star,numlines_in_file,r_arr);

#ifdef MAX
#undef MAX
#endif
#define MAX(A, B) ( ((A) > (B)) ? (A) : (B) )

  int idxmin = MAX(0,idx-interp_stencil_size/2-1);

  // Now perform the Lagrange polynomial interpolation:

  // First set the interpolation coefficients:
  REAL r_sample[interp_stencil_size];
  for(int i=idxmin;i<idxmin+interp_stencil_size;i++) {
    r_sample[i-idxmin] = r_arr[i];
  }
  REAL l_i_of_r[interp_stencil_size];
  for(int i=0;i<interp_stencil_size;i++) {
    REAL numer = 1.0;
    REAL denom = 1.0;
    for(int j=0;j<i;j++) {
      numer *= r_star - r_sample[j];
      denom *= r_sample[i] - r_sample[j];
    }
    for(int j=i+1;j<interp_stencil_size;j++) {
      numer *= r_star - r_sample[j];
      denom *= r_sample[i] - r_sample[j];
    }
    l_i_of_r[i] = numer/denom;
  }

  // Then perform the interpolation:
  *sf_star                = 0.0;
  *gammaDD00_or_psi4_star = 0.0;
  *alpha_star             = 0.0;

  for(int i=idxmin;i<idxmin+interp_stencil_size;i++) {
    *sf_star                += l_i_of_r[i-idxmin] * sf_arr[i];
    *gammaDD00_or_psi4_star += l_i_of_r[i-idxmin] * gammaDD00_or_psi4_arr[i];
    *alpha_star             += l_i_of_r[i-idxmin] * alpha_arr[i];
  }
}
"""
    
    preloop = r"""
  // Step 1: Set up scalar field initial data
  // Step 1.a: Read scalar field initial data from data file
  // Open the data file:
  char filename[100];
  sprintf(filename,"./SFID.txt");
  FILE *fp = fopen(filename, "r");
  if (fp == NULL) {
    fprintf(stderr,"ERROR: could not open file %s\n",filename);
    exit(1);
  }
  // Step 1.b: Count the number of lines in the data file:
  int numlines_in_file = count_num_lines_in_file(fp);

  // Step 1.c: Allocate memory for all data arrays:
  REAL *r_arr     = (REAL *)malloc(sizeof(REAL)*numlines_in_file);
  REAL *sf_arr    = (REAL *)malloc(sizeof(REAL)*numlines_in_file);
  REAL *psi4_arr  = (REAL *)malloc(sizeof(REAL)*numlines_in_file);
  REAL *alpha_arr = (REAL *)malloc(sizeof(REAL)*numlines_in_file);

  // Step 1.d: Read from datafile
  if(read_datafile__set_arrays(fp,r_arr,sf_arr,psi4_arr,alpha_arr) == 1) {
    fprintf(stderr,"ERROR WHEN READING FILE %s!\n",filename);
    exit(1);
  }
  fclose(fp);
"""
    body = outputC(rhss, lhss, filename="returnstring",
                   params="preindent=1,CSE_enable=True,outCverbose=False",  # outCverbose=False to prevent
                                                                            # enormous output files.
                   prestring="", poststring="")

    postloop = r"""
  free(r_arr);
  free(sf_arr);
  free(psi4_arr);
  free(alpha_arr);
"""

    desc = "Set up the initial data at all points on the numerical grid."
    add_to_Cfunction_dict(
        prefunc =prefunc,
        includes=includes,
        desc    =desc,
        name    ="initial_data",
        params  ="const paramstruct *restrict params,REAL *restrict xx[3], REAL *restrict in_gfs",
        preloop=preloop,
        body    =body,
        postloop=postloop,
        loopopts="AllPoints,Read_xxs")

ScalarField_InitialData()